In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import warnings
from sklearn.preprocessing import OneHotEncoder
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

traindataFile = u"E:/BDCI Match/train_preproceed.csv"

df = pd.read_csv(traindataFile)

#对预测标签重新编码
le = LabelEncoder()
#获取预测结果的所有取值
res_labels = df["current_service"].unique()

le.fit(res_labels)
df["res"] = le.transform(df["current_service"])

features_selected =  ["service_type","online_time","1_total_fee","2_total_fee", "3_total_fee", "4_total_fee",
                     "month_traffic", "many_over_bill", "contract_type","contract_time","pay_num","last_month_traffic",
                     "local_trafffic_month","local_caller_time","service1_caller_time","service2_caller_time", "age"]

param_grid = {
    "boosting_type" : ['gbdt','dart'],
    ""
}
muti_results = lgb.LGBMClassifier(
   ,
    num_leaves = 39,
    max_depth = 6,
    learning_rate = 0.02,
    n_estimators = 10000,
    max_bin = 255,
    subsample_for_bin = 200000,
    objective = 'multiclass',
    min_split_gain = 0.0,
    min_child_weight = 0.001,
    min_child_samples = 20,
    subsample = 1.0,
    subsample_freq = 1,
    colsample_bytree = 1.0,
    reg_alpha = 0.0,
    reg_lambda = 0.0,
    random_state = 0,
    n_jobs = 3,
    silent = True,
    early_stopping_rounds=200,
    verbose_eval=100,
    show_stdv=True)
muti_results.fit(X_train,
                 y_train,
                 eval_set=[(X_test, y_test)],
                 early_stopping_rounds = 200
                )

In [ ]:
lgb.create_tree_digraph(lgb_clf, tree_index=1)

In [ ]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 基础配置信息
path = 'E:/BDCI Match/'
n_splits = 5
seed = 42

# lgb 参数
params={
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":15,
    "silent":True,
}

# 读取数据
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

'''
简单分析数据：
user_id 为编码后的数据，大小：
train data shape (612652, 27)
train data of user_id shape 612652
简单的1个用户1条样本的题目,标签的范围 current_service
'''
print('标签',set(train.columns)-set(test.columns))

print('train data shape',train.shape)
print('train data of user_id shape',len(set(train['user_id'])))
print('train data of current_service shape',(set(train['current_service'])))

print('train data shape',test.shape)
print('train data of user_id shape',len(set(test['user_id'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)

features_selected =  ["service_type","online_time","1_total_fee","2_total_fee", "3_total_fee", "4_total_fee",
                     "month_traffic", "many_over_bill", "contract_type","contract_time","pay_num","last_month_traffic",
                     "local_trafffic_month","local_caller_time","service1_caller_time","service2_caller_time", "age"]

X = train[features_selected]
y = train.pop('current_service')
X_test = test[features_selected]
train_id = train.pop('user_id')
test_id = test['user_id']
train_col = train.columns

# 数据有问题数据
for i in features_selected:
    X[i] = X[i].replace("\\N",-1)
    X_test[i] = X_test[i].replace("\\N",-1)
    
# # 构造原始数据
# y = train.pop('current_service')
# train_id = train.pop('user_id')
# # 这个字段有点问题
# X = train
# train_col = train.columns

# X_test = test[train_col]
# test_id = test['user_id']

# # 数据有问题数据
# for i in train_col:
#     X[i] = X[i].replace("\\N",-1)
#     X_test[i] = X_test[i].replace("\\N",-1)

X,y,X_test = X.values,y,X_test.values

# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(15, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)

    clf=lgb.train(params,train_data,num_boost_round=100000,valid_sets=[validation_data],early_stopping_rounds=50,feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv('E:/BDCI Match/result/baseline2.csv',index=False)

print(xx_score,np.mean(xx_score))

In [10]:
clf.feature_importances_

AttributeError: 'Booster' object has no attribute 'feature_importances_'

In [ ]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(15, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

# 基础配置信息
path = 'E:/BDCI Match/'
n_splits = 5
seed = 42

# 读取数据
train = pd.read_csv(path + 'train.csv')

'''
简单分析数据：
user_id 为编码后的数据，大小：
train data shape (612652, 27)
train data of user_id shape 612652
简单的1个用户1条样本的题目,标签的范围 current_service
'''

print('train data shape',train.shape)
print('train data of user_id shape',len(set(train['user_id'])))
print('train data of current_service shape',(set(train['current_service'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)


# 构造原始数据
y = train.pop('current_service')
train_id = train.pop('user_id')
# 这个字段有点问题
X = train
train_col = train.columns

# 数据有问题数据
for i in train_col:
    X[i] = X[i].replace("\\N",-1)

X,y = X.values,y

lg = lgb.LGBMClassifier(
    max_bin = 255,
    subsample_for_bin = 200000,
    objective = 'multiclass',
    num_class =15,
    min_split_gain = 0.0,
    min_child_weight = 0.001,
    min_child_samples = 20,
    subsample = 1.0,
    subsample_freq = 1,
    reg_alpha = 0.0,
    reg_lambda = 0.0,
    random_state = 0
)

param_dict = {
    "boosting_type" : ['gbdt','dart'],
    "max_depth": [5, 8, 10, 20],
    "learning_rate": [0.01, 0.05, 0.1],
    "num_leaves": [40, 50, 100, 200],
    "n_estimators": [3000, 5000, 10000, 20000],
    "colsample_bytree": [0.8, 0.9, 1.0],
}

grid = GridSearchCV(lg, n_jobs=3, param_grid=param_dict, cv = 2, scoring='roc_auc', verbose=5)
                           
grid.fit(X,y)

bestModel = grid.best_estimator_

bestParam = grid.best_params_ 

print("bestParam; ")
print(bestParam)

bestScore = grid.best_score_ 

print("bestScore; ")
print(bestScore)

bestModel.save_model('./Model.txt')

# 保存结果
df_test = pd.DataFrame()

for key in bestParam.keys():
    t = []
    t.append(bestParam[key])
    df_test[key] = t

df_test.to_csv('./Para.csv',index=False)

train data shape (612652, 27)
train data of user_id shape 612652
train data of current_service shape {89016259, 89016252, 90155946, 99999825, 90063345, 99104722, 99999828, 99999826, 89950166, 89950167, 89950168, 99999827, 99999830, 90109916, 89016253}
Fitting 2 folds for each of 1152 candidates, totalling 2304 fits


In [20]:
import pandas as pd

bestParam = {"A":0, "B":1, "C":2}

df_test = pd.DataFrame()

print(bestParam)


df_test.to_csv('E:/BDCI Match/result/test.csv',index=False)


{'A': 0, 'B': 1, 'C': 2}


TypeError: 'int' object is not iterable

In [ ]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# 基础配置信息
path = 'E:/BDCI Match/'
n_splits = 5
seed = 42

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(15, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

# 读取数据
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

# '''
# 简单分析数据：
# user_id 为编码后的数据，大小：
# train data shape (612652, 27)
# train data of user_id shape 612652
# 简单的1个用户1条样本的题目,标签的范围 current_service
# '''
# print('标签',set(train.columns)-set(test.columns))

# print('train data shape',train.shape)
# print('train data of user_id shape',len(set(train['user_id'])))
# print('train data of current_service shape',(set(train['current_service'])))

# print('train data shape',test.shape)
# print('train data of user_id shape',len(set(test['user_id'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)
    
# 构造原始数据
y = train.pop('current_service')
train_id = train.pop('user_id')
# 这个字段有点问题
X = train
train_col = train.columns

X_test = test[train_col]
test_id = test['user_id']

# 数据有问题数据
for i in train_col:
    X[i] = X[i].replace("\\N",-1)
    X_test[i] = X_test[i].replace("\\N",-1)

X,y,X_test = X.values,y,X_test.values

#数据拆分
print('数据拆分')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=21)

### 数据转换
print('数据转换')
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

### 设置初始参数--不含交叉验证参数
print('设置参数')
params = {
            'boosting_type': 'gbdt',
            'objective' : 'multiclass',
            'num_class' : 15,
            'num_boost_round' : 3000,
            'metric': 'binary_logloss',
          }


### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# 准确率
print("调参1：提高准确率")
for num_leaves in range(20,40):
    for max_depth in range(3,6):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth

        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=201,
                            nfold=3,
                            metrics=['binary_logloss'],
                            early_stopping_rounds=20,
                            verbose_eval=150,
                            show_stdv = True
                            )
            
        mean_merror = pd.Series(cv_results['binary_logloss-mean']).min()
        boost_rounds = pd.Series(cv_results['binary_logloss-mean']).argmin()
            
        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
            
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']

# 过拟合
print("调参2：降低过拟合")
for max_bin in range(5,255):
    for min_data_in_leaf in range(10,200):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                metrics=['binary_logloss'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_merror = pd.Series(cv_results['binary_logloss-mean']).min()
            boost_rounds = pd.Series(cv_results['binary_logloss-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf

params['min_data_in_leaf'] = best_params['min_data_in_leaf']
params['max_bin'] = best_params['max_bin']

print("调参3：降低过拟合")
for feature_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                metrics=['binary_logloss'],
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                    
            mean_merror = pd.Series(cv_results['binary_logloss-mean']).min()
            boost_rounds = pd.Series(cv_results['binary_logloss-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

params['feature_fraction'] = best_params['feature_fraction']
params['bagging_fraction'] = best_params['bagging_fraction']
params['bagging_freq'] = best_params['bagging_freq']

print("调参4：降低过拟合")
for lambda_l1 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    for lambda_l2 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            params['lambda_l1'] = lambda_l1
            params['lambda_l2'] = lambda_l2
            params['min_split_gain'] = min_split_gain
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                metrics=['binary_logloss'],
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                    
            mean_merror = pd.Series(cv_results['binary_logloss-mean']).min()
            boost_rounds = pd.Series(cv_results['binary_logloss-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['lambda_l1'] = lambda_l1
                best_params['lambda_l2'] = lambda_l2
                best_params['min_split_gain'] = min_split_gain

params['lambda_l1'] = best_params['lambda_l1']
params['lambda_l2'] = best_params['lambda_l2']
params['min_split_gain'] = best_params['min_split_gain']


print(best_params)

### 训练
params['learning_rate']=0.01
lgb.train(
          params,                     # 参数字典
          lgb_train,                  # 训练集
          valid_sets=lgb_eval,        # 验证集
          num_boost_round=2000,       # 迭代次数
          early_stopping_rounds=50    # 早停次数
          )

### 保存模型
from sklearn.externals import joblib
joblib.dump(lgb,path +'lgb.pkl')

### 特征选择
df = pd.DataFrame(X_train.columns.tolist(), columns=['feature'])
df['importance']=list(lgb.feature_importance())                           # 特征分数
df = df.sort_values(by='importance',ascending=False)                      # 特征排序
df.to_csv(path +"feature_score_20180919.csv",index=None,encoding='gbk')  # 保存分数


# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)

    clf=lgb.train(params,train_data,num_boost_round=100000,valid_sets=[validation_data],early_stopping_rounds=50,feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv(path +'result/baseline2.csv',index=False)